In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from urllib.request import urlopen
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import requests

### a function to replace multiple portions (in the form of a list) of string by one common string

In [2]:
def replaceMultiple(mainString, toBeReplaces, newString):
    # Iterate over the strings to be replaced
    for elem in toBeReplaces :
        # Check if string is in the main string
        if elem in mainString :
            # Replace the string
            mainString = mainString.replace(elem, newString)
    
    return  mainString

In [3]:
# open the two spread sheets in data frames
df_county=pd.read_excel('data_born_pol.xlsx', index_col=0)
df_rel = pd.read_excel('data_state_county_religion_final.xlsx', index_col=0)

In [39]:
# a dictionary for data entry
dic_final={}

# count for dictionary entry
count = 0

not_found = {}
n1 = 0

# loop over each politician name in the religion dataframe
for each in df_rel['pol_name']:

    match_found = 0

    # flag identifies if the name has not yet been matched in the county dataframe
    flag = 0

    # store different parts of the name in a list after removing the titles
    temp_name1 = replaceMultiple(each, [", Jr.", ", Sr."],"").split(' ')

    # store the name of the religion of that politician
    
    res_st = df_rel.loc[df_rel['pol_name']==each].iloc[0,1]
    born_st = df_rel.loc[df_rel['pol_name']==each].iloc[0,2]
    born_county = df_rel.loc[df_rel['pol_name']==each].iloc[0,3]
    rel = df_rel.loc[df_rel['pol_name']==each].iloc[0,4]

    df_sub = df_county[df_county['state'] == born_st][df_county['county'] == born_county]

    # if a name has not yet been matched
    if(flag == 0):
        # loop over the politician names in county dataframe
        for item in df_sub['pol_name']:

            # identifies the number of word in the name that have been matched
            id = 1

            # if a name has not yet been matched
            if(flag == 0):

                # store different parts of the name in a list after removing the titles
                temp_name2 = replaceMultiple(item, [", Jr.", ", Jr. ", ", Sr.", ", Sr. "],"").split(' ')

                # loop over each word in the politician name from religion dataframe
                for i in np.arange(len(temp_name1)):

                    # if a name has not yet been matched
                    if(flag == 0):

                            # loop over each word in the politician name from religion dataframe
                            for j in np.arange(len(temp_name2)):
                                
                                # if the words match and the words do not contain abbreviated portions
                                if((temp_name1[i] == temp_name2[j]) & (('.' in temp_name1[i]) == False) & (('.' in temp_name2[j]) == False)):

                                    # if two words in the name have been matched 
                                    if(id==2):

                                        # find the corresponding county name
                                        #birth_county = df_sub.loc[df_county['pol_name']==item].iloc[0,1]

                                        # find the corresponding state name
                                        #birth_st = df_sub.loc[df_county['pol_name']==item].iloc[0,0]

                                        # save the data as a new dictionary entry
                                        dic_final[count] = {'residence state': res_st, 'birth state': born_st, 'birth_county': born_county, 'pol_name_religion': each, 'pol_name_county': item, 'religion': rel }
                                        # a name was matched and now the code should jump to the outer most loop
                                        flag = 1 
                                        match_found = 1

                                        # increase the counter
                                        count += 1

                                        # reset the id to 1
                                        id = 1

                                        break

                                    else:
                                        # if words match for the first time, increase value of id
                                        id += 1

    if match_found == 0:
        not_found[n1] = {'residence state': res_st, 'birth state': born_st, 'birth_county': born_county, 'pol_name_religion': each, 'religion': rel }
        n1+=1

In [40]:
# convert dictionary to dataframe
df_final=pd.DataFrame.from_dict(dic_final).T
df_unmatched =  pd.DataFrame.from_dict(not_found).T

# convert dataframe to excel file
df_final.to_excel('data_name_match.xlsx',engine='xlsxwriter')
df_unmatched.to_excel('data_name_no_match.xlsx',engine='xlsxwriter')

15579

In [32]:
df_county[df_county['state'] == born_st][df_county['county'] == "Madison County"]

,state,county,pol_name
277,Alabama,Madison County,Edward A. O'Neal
278,Alabama,Madison County,Enoch H. Vance
279,Alabama,Madison County,David D. Shelby
280,Alabama,Madison County,Arthur Cartwright
281,Alabama,Madison County,Percy Walker
282,Alabama,Madison County,Jeremiah Clemens
283,Alabama,Madison County,Andrew Jackson Hamilton
284,Alabama,Madison County,"Clement Claiborne Clay, Jr."
285,Alabama,Madison County,Leroy P. Walker
286,Alabama,Madison County,James Phelan


In [8]:
df_rel

,Unnamed: 0.1,lived_state,born_state,born_county,religion,pol_name
0,0,Alabama,Alabama,Jefferson County,Methodist,"Oscar William Adams, Jr."
1,1,Alabama,Alabama,Winston County,Methodist,Robert Brown Aderholt
2,2,Alabama,Alabama,Blount County,Methodist,Miles Clayton Allgood
3,3,Alabama,Alabama,Lawrence County,Methodist,Edward Berton Almon
4,4,Alabama,Alabama,Mobile County,Methodist,Julian Leigh Andrews
...,...,...,...,...,...,...
20176,20176,none,second cousin five times removed of,Lithgow Osborne,Deist,Benjamin Franklin
20177,20177,none,Virginia,Albemarle County,Deist,Thomas Jefferson
20178,20178,none,Michigan,Wayne County,Scientologist,Sonny Bono
20179,20179,none,Delaware,New Castle County,Scientologist,Stephen A. Davis


In [9]:
df_county

,state,county,pol_name
0,Alabama,Autauga County,Rush Elmore
1,Alabama,Autauga County,Marcus H. Cruikshank
2,Alabama,Autauga County,Malcolm Daniel Graham
3,Alabama,Autauga County,Douglas Booth
4,Alabama,Autauga County,Daniel H. Thomas
...,...,...,...
41996,Wisconsin,Miscellaneous Areas,Leo J. Fox
41997,Wisconsin,Miscellaneous Areas,Richard J. Greuel
41998,Wisconsin,Miscellaneous Areas,Jalmar M. Kerttula
41999,Wisconsin,Miscellaneous Areas,Joan Heggen
